# LeNet

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
print(f'Import PyTorch V{torch.__version__}')

dev = torch.device(type='cuda') if torch.cuda.is_available() else torch.device(type='cpu')
print(f'Use device {dev}')

Import PyTorch V1.12.1
Use device cpu


In [4]:
print('Create LeNet Model')

def gen_lenet() -> torch.nn.Module:
    return torch.nn.Sequential(
        torch.nn.Conv2d(1, 6, kernel_size=5, padding=2), torch.nn.Sigmoid(),
        torch.nn.AvgPool2d(kernel_size=2, stride=2),
        torch.nn.Conv2d(6, 16, kernel_size=5), torch.nn.Sigmoid(),
        torch.nn.AvgPool2d(kernel_size=2, stride=2),
        torch.nn.Flatten(),
        torch.nn.Linear(16 * 5 * 5, 120), torch.nn.Sigmoid(),
        torch.nn.Linear(120, 84), torch.nn.Sigmoid(),
        torch.nn.Linear(84, 10), torch.nn.Softmax(),
    )

m = gen_lenet().to(dev)
print(m)


Create LeNet Model
Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): Sigmoid()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): Sigmoid()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=400, out_features=120, bias=True)
  (8): Sigmoid()
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): Sigmoid()
  (11): Linear(in_features=84, out_features=10, bias=True)
  (12): Softmax(dim=None)
)


In [7]:
print('Prepare fasion mnist dataset')

import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

batch_size = 32

ds_train = torchvision.datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

ds_test = torchvision.datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

loader_train = DataLoader(ds_train, batch_size, True)
loader_test  = DataLoader(ds_test,  batch_size)

for X, y in loader_test:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Prepare fasion mnist dataset
Using downloaded and verified file: data/FashionMNIST/raw/train-images-idx3-ubyte.gz
Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw

Using downloaded and verified file: data/FashionMNIST/raw/train-labels-idx1-ubyte.gz
Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Using downloaded and verified file: data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz
Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw

Using downloaded and verified file: data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Shape of X [N, C, H, W]: torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]) torch.int64


In [18]:
loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.SGD(m.parameters(), lr=0.9)
print(f'Prepared loss function {loss}')
print(f'Prepared optimizator {opt}')

Prepared loss function CrossEntropyLoss()
Prepared optimizator SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.9
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [12]:
from typing import Callable, Tuple

def train(
        dataloader: DataLoader,
        model: torch.nn.Module,
        loss: Callable,
        optimizer: torch.optim.Optimizer,
        print_cycle: int = 100,
) -> None:
    if not isinstance(dataloader, DataLoader) or \
            not isinstance(model, torch.nn.Module) or \
            not isinstance(loss, Callable) or \
            not isinstance(optimizer, torch.optim.Optimizer) or \
            not isinstance(print_cycle, int) or \
            print_cycle < 10:
        raise TypeError(f'Wrong input types: {type(dataloader)}, '
                        f'{type(model)}, {type(loss)}, {type(optimizer)}, '
                        f'{type(print_cycle)}, {print_cycle}')
    size = len(dataloader.dataset)
    model.train()
    dev = next(iter(model.parameters())).device
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
        X, y = X.to(dev), y.to(dev)
        # Compute prediction error
        pred = model(X)
        l = loss(pred, y)
        # Backpropagation
        l.backward()
        optimizer.step()
        if batch % print_cycle == 0:
            l, current = l.item(), (batch + 1) * len(X)
            print(f"loss: {l:>7f}  [{current:>5d}/{size:>5d}]")

def test(
        dataloader: DataLoader,
        model: torch.nn.Module,
        loss: Callable,
) -> Tuple[float, float]:
    if not isinstance(dataloader, DataLoader) or \
            not isinstance(model, torch.nn.Module) or \
            not isinstance(loss, Callable):
        raise TypeError(f'Wrong input types: {type(dataloader)}, '
                        f'{type(model)}, {type(loss)}')
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0.0, 0.0
    dev = next(iter(model.parameters())).device
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(dev), y.to(dev)
            pred = model(X)
            test_loss += loss(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return (test_loss, correct)

In [17]:
def init_weights(m):
    if type(m) == torch.nn.Linear or type(m) == torch.nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

m.apply(init_weights)

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): Sigmoid()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): Sigmoid()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=400, out_features=120, bias=True)
  (8): Sigmoid()
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): Sigmoid()
  (11): Linear(in_features=84, out_features=10, bias=True)
  (12): Softmax(dim=None)
)

In [20]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(loader_train, m, loss, opt, 600)
    l, correct = test(loader_test, m, loss)
    print(f'Test Error: Accuracy {(100*correct):>0.1f}% Avg loss {l:>8f}')
    print()

Epoch 1
-------------------------------


/Users/hzhou/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss: 1.624954  [   32/60000]
loss: 1.799572  [19232/60000]
loss: 1.681514  [38432/60000]
loss: 1.617116  [57632/60000]
Test Error: Accuracy 80.2% Avg loss 1.659420

Epoch 2
-------------------------------
loss: 1.639743  [   32/60000]
loss: 1.595454  [19232/60000]
loss: 1.711271  [38432/60000]
loss: 1.641790  [57632/60000]
Test Error: Accuracy 80.3% Avg loss 1.657520

Epoch 3
-------------------------------
loss: 1.627110  [   32/60000]
loss: 1.534080  [19232/60000]
loss: 1.686262  [38432/60000]
loss: 1.675175  [57632/60000]
Test Error: Accuracy 80.3% Avg loss 1.657564

Epoch 4
-------------------------------
loss: 1.541423  [   32/60000]
loss: 1.706483  [19232/60000]
loss: 1.557345  [38432/60000]
loss: 1.562943  [57632/60000]
Test Error: Accuracy 81.5% Avg loss 1.646560

Epoch 5
-------------------------------
loss: 1.568389  [   32/60000]
loss: 1.561717  [19232/60000]
loss: 1.709478  [38432/60000]
loss: 1.585127  [57632/60000]
Test Error: Accuracy 81.3% Avg loss 1.647421

